# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [8]:
df = pd.read_pickle("shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


***
### Show some exploratory data analysis here
***

In [3]:
df.columns
#This code allows us to  see what columns are in the dataframe

Index(['CRASH DATE_CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')

In [4]:
df['YEAR'] = df['CRASH DATE_CRASH TIME'].dt.year
df.groupby('YEAR')['CRASH DATE_CRASH TIME'].count()
#This code allows us to see what years the dataframe contains

YEAR
2012    100545
2013    203738
2014    206033
2015    217694
2016    229831
2017    231007
2018    231564
2019    211486
2020    112915
2021    110548
2022    103865
2023     59019
Name: CRASH DATE_CRASH TIME, dtype: int64

## Borough with the maximum number of crashes reported since 2012

In [5]:
df.groupby('BOROUGH')['COLLISION_ID'].count().sort_values(ascending=False)
#This code allows us to filter the number of crashes by borough

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: COLLISION_ID, dtype: int64

In [6]:
#The result shows us Brooklyn has had the most number of crashes reported since 2012.

In [7]:
df['BOROUGH'].isna().sum()
#This just shows us how many reports did not list the borough

627854

## Borough with the maximum number of crashes adjusted for population

In [17]:
pop = pd.DataFrame(data = {'BOROUGH': ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
                           'POPULATION': [1446788, 2648452, 1638281, 2330295, 487155], 
             },) 

pop

,BOROUGH,POPULATION
0,BRONX,1446788
1,BROOKLYN,2648452
2,MANHATTAN,1638281
3,QUEENS,2330295
4,STATEN ISLAND,487155


In [18]:
df1 = df.merge(pop,how='left', left_on='BOROUGH', right_on='BOROUGH')

In [19]:
df1

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,POPULATION
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,2648452.0
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,2648452.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN,1446788.0
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN,1638281.0
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,2330295.0


In [74]:
df.groupby('BOROUGH')['COLLISION_ID'].count()

BOROUGH
BRONX            205345
BROOKLYN         441026
MANHATTAN        313266
QUEENS           372457
STATEN ISLAND     58297
Name: COLLISION_ID, dtype: int64

In [72]:
df1.groupby('BOROUGH')['POPULATION'].first()

BOROUGH
BRONX            1446788.0
BROOKLYN         2648452.0
MANHATTAN        1638281.0
QUEENS           2330295.0
STATEN ISLAND     487155.0
Name: POPULATION, dtype: float64

In [76]:
((df.groupby('BOROUGH')['COLLISION_ID'].count() / df1.groupby('BOROUGH')['POPULATION'].first()) * 100000).sort_values(ascending=False)

BOROUGH
MANHATTAN        19121.628097
BROOKLYN         16652.217975
QUEENS           15983.255339
BRONX            14193.164444
STATEN ISLAND    11966.827806
dtype: float64

## Analyzing the leading cause of crashes

In [154]:
remove = "Unspecified"

# Create a new Series excluding the row with the value to remove
factor1 = df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().sort_values(ascending=False)
factor1 = factor1[factor1.index != remove]
(factor1.head(1) / factor1.sum())*100

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction    30.41797
Name: count, dtype: float64

## Top 3 causes of crashes 

In [151]:
factor1.head(3).sum() /factor1.sum()

0.4759805664357876

## Analyzing fatalities

In [113]:
df['NUMBER OF PERSONS KILLED'].value_counts()

NUMBER OF PERSONS KILLED
0.0    2015410
1.0       2716
2.0         71
3.0         12
4.0          3
8.0          1
5.0          1
Name: count, dtype: int64

In [114]:
total_crashes_with_killed = (df['NUMBER OF PERSONS KILLED'] >= 1).sum()

print("Total Crashes with at Least 1 Person Killed:", total_crashes_with_killed)

Total Crashes with at Least 1 Person Killed: 2804


## Likelihood of fatal accidents
#### On average, out of every 1000 accidents, how many have resulted in at least one person dead?

In [115]:
total_accidents = len(df)  # Total number of accidents
total_accidents_with_killed = (df['NUMBER OF PERSONS KILLED'] >= 1).sum()  # Total number of accidents with at least 1 person killed

# Calculate the average per 1000 accidents
average_per_1000_accidents = (total_accidents_with_killed / total_accidents) * 1000

print("On average, out of every 1000 accidents, approximately", average_per_1000_accidents, "result in at least one person dead.")

On average, out of every 1000 accidents, approximately 1.3893258747079764 result in at least one person dead.


## Missing data
#### What proportion of accidents in the data do not have a Borough code?

In [153]:
missing_borough_count = df['BOROUGH'].isna().sum()

total_accidents = len(df)

proportion_without_borough = (missing_borough_count / total_accidents) * 100

print("Proportion of accidents without a Borough code:", proportion_without_borough)

Proportion of accidents without a Borough code: 31.10890897784957


In [117]:
df['BOROUGH'].isna().sum()

627854

## Which combinations of vehicles have the most number of accidents

In [139]:
vehicle_columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']
vehicle = df[vehicle_columns].value_counts()
vehicle.head(10)

VEHICLE TYPE CODE 1                  VEHICLE TYPE CODE 2                
Sedan                                Sedan                                  197944
PASSENGER VEHICLE                    PASSENGER VEHICLE                      193260
Station Wagon/Sport Utility Vehicle  Station Wagon/Sport Utility Vehicle    133780
Sedan                                Station Wagon/Sport Utility Vehicle    123915
Station Wagon/Sport Utility Vehicle  Sedan                                  123812
PASSENGER VEHICLE                    SPORT UTILITY / STATION WAGON           63053
SPORT UTILITY / STATION WAGON        PASSENGER VEHICLE                       62982
                                     SPORT UTILITY / STATION WAGON           50156
PASSENGER VEHICLE                    UNKNOWN                                 49073
SPORT UTILITY / STATION WAGON        UNKNOWN                                 19412
Name: count, dtype: int64

***  
## Finished